# 1 - Set Up

This notebook goes over how to set up the necessary cloud resources to run the RAG framework. You will be performing the following steps:

1. Creating a GCP bucket
2. Creating a VertexAI Vector Search Index
3. Creating a VertexAI Vector Search endpoint
4. Deploying a VertexAI Vector Search endpoint

## Imports

Install the following modules to run this notebook.

In [ ]:
!pip install google-cloud-aiplatform \
  google-cloud-storage 

In [1]:
import os
from google.cloud import storage
from google.cloud import aiplatform

### Download Data

In [ ]:
!git clone https://github.com/noabenefraim/qwiklabs_data.git data

## Enable Vertex API and set API Key

Before you continue, make sure to refer to the lab instructions to do the follow:

1. Enable VertexAI API
2. Set a GOOGLE_API_KEY 

Note, you will use the GOOGLE_API_KEY in the next lab notebook.

## Create resources

In [3]:
PROJECT_ID = "" #TODO - add your project-id here from the console
REGION = ""  #TODO - add your region here from the console
GCS_BUCKET = "llamaindex_gcs_bucket"  # @param {type:"string"}
VS_INDEX_NAME = "llamaindex_doc_index"  # @param {type:"string"}
VS_INDEX_ENDPOINT_NAME = "llamaindex_doc_endpoint"  # @param {type:"string"}
DOC_FOLDER = "./data"  # @param {type:"string"}

In [9]:
def create_bucket_class_location(bucket_name):
    """
    Create a new bucket in the US region with the coldline storage
    class.
    """

    storage_client = storage.Client()

    #Searching for existing GCS bucket
    for bucket in storage_client.list_buckets():
        if bucket.name==bucket_name:
            print(
                f"GCS Bucket {bucket_name} exists already in resource."
            )
            return bucket

    #Creating new bucket
    bucket = storage_client.bucket(bucket_name)
    bucket.storage_class = "STANDARD"
    new_bucket = storage_client.create_bucket(bucket, location=REGION)

    print(
        "Created bucket {} in {} with storage class {}".format(
            new_bucket.name, new_bucket.location, new_bucket.storage_class
        )
    )

    return new_bucket

def create_vertex_ai_search_index(index_name, index_dimensions):
    """
    Creates a VertexAI Search Index
    NOTE : This operation can take upto 30 minutes
    """

    # check if index exists
    index_names = [
        index.resource_name
        for index in aiplatform.MatchingEngineIndex.list(
            filter=f"display_name={index_name}"
        )
    ]

    if len(index_names) == 0:
        print(f"Creating Vector Search index {index_name} ...")
        vs_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
            display_name=index_name,
            dimensions=index_dimensions,
            distance_measure_type="DOT_PRODUCT_DISTANCE",
            shard_size="SHARD_SIZE_SMALL",
            index_update_method="STREAM_UPDATE",  # allowed values BATCH_UPDATE , STREAM_UPDATE,
            approximate_neighbors_count = 5
        )
        print(
            f"Vector Search index {vs_index.display_name} created with resource name {vs_index.resource_name}"
        )
    else:
        vs_index = aiplatform.MatchingEngineIndex(index_name=index_names[0])
        print(
            f"Vector Search index {vs_index.display_name} exists with resource name {vs_index.resource_name}"
        )

    return vs_index

def create_vertexai_search_endpoint(endpoint_name):
    """
    Creates a VertexAI Search endpoint.
    """
    endpoint_names = [
        endpoint.resource_name
        for endpoint in aiplatform.MatchingEngineIndexEndpoint.list(
            filter=f"display_name={endpoint_name}"
        )
    ]

    if len(endpoint_names) == 0:
        print(
            f"Creating Vector Search index endpoint {endpoint_name} ..."
        )
        vs_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
            display_name=endpoint_name, public_endpoint_enabled=True
        )
        print(
            f"Vector Search index endpoint {vs_endpoint.display_name} created with resource name {vs_endpoint.resource_name}"
        )
    else:
        vs_endpoint = aiplatform.MatchingEngineIndexEndpoint(
            index_endpoint_name=endpoint_names[0]
        )
        print(
            f"Vector Search index endpoint {vs_endpoint.display_name} exists with resource name {vs_endpoint.resource_name}"
        )

    return vs_endpoint

def deploy_vertexai_search_endpoint(vs_index, vs_endpoint, index_name):
    """
    Deploys a VertexAI search endpoint.
    """
    # check if endpoint exists
    index_endpoints = [
        (deployed_index.index_endpoint, deployed_index.deployed_index_id)
        for deployed_index in vs_index.deployed_indexes
    ]

    if len(index_endpoints) == 0:
        print(
            f"Deploying Vector Search index {vs_index.display_name} at endpoint {vs_endpoint.display_name} ..."
        )
        vs_deployed_index = vs_endpoint.deploy_index(
            index=vs_index,
            deployed_index_id=index_name,
            display_name=index_name,
            machine_type="e2-standard-16",
            min_replica_count=1,
            max_replica_count=1,
        )
        print(
            f"Vector Search index {vs_index.display_name} is deployed at endpoint {vs_deployed_index.display_name}"
        )
    else:
        vs_deployed_index = aiplatform.MatchingEngineIndexEndpoint(
            index_endpoint_name=index_endpoints[0][0]
        )
        print(
            f"Vector Search index {vs_index.display_name} is already deployed at endpoint {vs_deployed_index.display_name}"
        )

    return vs_deployed_index

In [10]:
def setup():
    GCS_BUCKET_URI = f"gs://{GCS_BUCKET}"

    # The number of dimensions for the gecko text embeddings is 768
    VS_DIMENSIONS = 768
    # Vertex AI Vector Search Index configuration

    aiplatform.init(project=PROJECT_ID, location=REGION)

    new_bucket = create_bucket_class_location(GCS_BUCKET)
    vs_index = create_vertex_ai_search_index(VS_INDEX_NAME, VS_DIMENSIONS)
    vs_endpoint = create_vertexai_search_endpoint(VS_INDEX_ENDPOINT_NAME)
    vs_deployed_index = deploy_vertexai_search_endpoint(vs_index, vs_endpoint, VS_INDEX_NAME)

    return new_bucket, vs_index, vs_endpoint, vs_deployed_index


In [ ]:
# This step can take ~30 minutes to set up the necessary resources.
bucket,vs_index, vs_endpoint, deployed_endpoint = setup()


Congratulations! Set up is complete. 

Run the cell below to capture the IDs of the resources that we will use in the next notebook.

In [ ]:
print("Index ID: " + vs_index.name)
print("Endpoint ID: " + vs_endpoint.name)